In [ ]:
import sys
sys.path.insert(0, "../")


import random
from copy import deepcopy

import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from tqdm import tqdm

from src.dataloader import *
from triage.triage import Triage
from src.utils import *

In [2]:
df, _, _ = load_seer_cutract_dataset(name="seer", seed=42)

cols = df.columns
long_tail = False
imbalance = False

In [3]:
def evaluate_methods(
    X_prop_train,
    y_prop_train,
    X_cal,
    y_cal,
    X_test,
    y_test,
    seed,
    nest,
    groups_id,
    long_tail=False,
    imbalance=False,
):
    from sklearn.metrics import mean_absolute_error as reg_metric
    from sklearn.metrics import mean_squared_error as reg_metric_mse
    from sklearn.linear_model import LinearRegression

    results = {}
    results_mse = {}

    extra_eval = False

    if imbalance == True or long_tail == True:
        results_mse_above = {}
        results_mse_below = {}
        extra_eval = True

        if imbalance == True:
            for i in range(len(cols)):
                if cols[i] == "age":
                    idx = i

                above_ids = np.where(X_test[:, idx] >= 65)[0]
                below_ids = np.where(X_test[:, idx] < 65)[0]

        elif long_tail == True:
            y_test > 0.75
            above_ids = np.where(y_test > 0.75)[0]
            below_ids = np.where(y_test < 0.75)[0]

    ######
    print("TRIAGE WE...")
    myids = groups_id["we_group"]
    keep = len(myids)
    print(f"{cal_size} - keep is {keep}")

    learner_prop = xgb.XGBRegressor(n_estimators=nest, random_state=seed)
    learner_prop.fit(X_prop_train[myids, :], y_prop_train[myids])

    y_pred = learner_prop.predict(X_test)

    results["TRIAGE_WE"] = reg_metric(y_test, y_pred)
    results_mse["TRIAGE_WE"] = reg_metric_mse(y_test, y_pred)

    if extra_eval == True:
        results_mse_above["TRIAGE_WE"] = reg_metric_mse(
            y_test[above_ids], y_pred[above_ids]
        )
        results_mse_below["TRIAGE_WE"] = reg_metric_mse(
            y_test[below_ids], y_pred[below_ids]
        )

    ######
    print("TRIAGE OE ...")
    myids = groups_id["oe_group"]

    learner_prop = xgb.XGBRegressor(n_estimators=nest, random_state=seed)
    learner_prop.fit(X_prop_train[myids, :], y_prop_train[myids])

    y_pred = learner_prop.predict(X_test)

    results["TRIAGE_OE"] = reg_metric(y_test, y_pred)
    results_mse["TRIAGE_OE"] = reg_metric_mse(y_test, y_pred)

    if extra_eval == True:
        results_mse_above["TRIAGE_OE"] = reg_metric_mse(
            y_test[above_ids], y_pred[above_ids]
        )
        results_mse_below["TRIAGE_OE"] = reg_metric_mse(
            y_test[below_ids], y_pred[below_ids]
        )

    ######
    print("TRIAGE UE...")
    myids = groups_id["ue_group"]

    learner_prop = xgb.XGBRegressor(n_estimators=nest, random_state=seed)
    learner_prop.fit(X_prop_train[myids, :], y_prop_train[myids])

    y_pred = learner_prop.predict(X_test)

    results["TRIAGE_UE"] = reg_metric(y_test, y_pred)
    results_mse["TRIAGE_UE"] = reg_metric_mse(y_test, y_pred)

    if extra_eval == True:
        results_mse_above["TRIAGE_UE"] = reg_metric_mse(
            y_test[above_ids], y_pred[above_ids]
        )
        results_mse_below["TRIAGE_UE"] = reg_metric_mse(
            y_test[below_ids], y_pred[below_ids]
        )

    ######
    print("TRIAGE Combined...")
    myids = np.hstack(
        (np.array(groups_id["ue_group"]), np.array(groups_id["oe_group"]))
    )

    learner_prop = xgb.XGBRegressor(n_estimators=nest, random_state=seed)
    learner_prop.fit(X_prop_train[myids, :], y_prop_train[myids])

    y_pred = learner_prop.predict(X_test)

    results["TRIAGE_Combined"] = reg_metric(y_test, y_pred)
    results_mse["TRIAGE_Combined"] = reg_metric_mse(y_test, y_pred)

    if extra_eval == True:
        results_mse_above["TRIAGE_Combined"] = reg_metric_mse(
            y_test[above_ids], y_pred[above_ids]
        )
        results_mse_below["TRIAGE_Combined"] = reg_metric_mse(
            y_test[below_ids], y_pred[below_ids]
        )

    #######
    print("CPS CONFIDENCE...")
    confidence = groups_id["confidence"]
    myids = np.where((confidence < 0.8) & (confidence > 0.2))[0]

    learner_prop = xgb.XGBRegressor(n_estimators=nest, random_state=seed)
    learner_prop.fit(X_prop_train[myids, :], y_prop_train[myids])

    y_pred = learner_prop.predict(X_test)

    results["Confidence"] = reg_metric(y_test, y_pred)
    results_mse["Confidence"] = reg_metric_mse(y_test, y_pred)

    if extra_eval == True:
        results_mse_above["Confidence"] = reg_metric_mse(
            y_test[above_ids], y_pred[above_ids]
        )
        results_mse_below["Confidence"] = reg_metric_mse(
            y_test[below_ids], y_pred[below_ids]
        )

    #####
    print("RESIDUALS DIRECT...")
    errors_array = groups_id["errors_array"]
    myids = np.argsort(np.abs(np.mean(errors_array, axis=-1)))[0:keep]

    myids = np.argsort(np.abs(errors_array)[:, -1])[0:keep]

    learner_prop = xgb.XGBRegressor(n_estimators=nest, random_state=seed)
    learner_prop.fit(X_prop_train[myids, :], y_prop_train[myids])

    y_pred = learner_prop.predict(X_test)

    results["Residuals_direct"] = reg_metric(y_test, y_pred)
    results_mse["Residuals_direct"] = reg_metric_mse(y_test, y_pred)

    if extra_eval == True:
        results_mse_above["Residuals_direct"] = reg_metric_mse(
            y_test[above_ids], y_pred[above_ids]
        )
        results_mse_below["Residuals_direct"] = reg_metric_mse(
            y_test[below_ids], y_pred[below_ids]
        )

    #####
    print("INTERVALS DIRECT...")
    interval_array = groups_id["interval_array"]
    # myids = np.argsort(np.abs(np.mean(interval_array, axis=-1)))[0:keep]
    myids = np.argsort(np.abs(interval_array)[:, -1])[0:keep]

    learner_prop = xgb.XGBRegressor(n_estimators=nest, random_state=seed)
    learner_prop.fit(X_prop_train[myids, :], y_prop_train[myids])

    y_pred = learner_prop.predict(X_test)

    results["Intervals_direct"] = reg_metric(y_test, y_pred)
    results_mse["Intervals_direct"] = reg_metric_mse(y_test, y_pred)

    if extra_eval == True:
        results_mse_above["Intervals_direct"] = reg_metric_mse(
            y_test[above_ids], y_pred[above_ids]
        )
        results_mse_below["Intervals_direct"] = reg_metric_mse(
            y_test[below_ids], y_pred[below_ids]
        )

    ####
    print("Actual eval fit...")
    learner_prop = xgb.XGBRegressor(n_estimators=nest, random_state=seed)
    learner_prop.fit(X_cal, y_cal)

    y_pred = learner_prop.predict(X_test)

    results["True_eval_fit"] = reg_metric(y_test, y_pred)
    results_mse["True_eval_fit"] = reg_metric_mse(y_test, y_pred)

    if extra_eval == True:
        results_mse_above["True_eval_fit"] = reg_metric_mse(
            y_test[above_ids], y_pred[above_ids]
        )
        results_mse_below["True_eval_fit"] = reg_metric_mse(
            y_test[below_ids], y_pred[below_ids]
        )

    ####
    print("MIX fit...")
    learner_prop = xgb.XGBRegressor(n_estimators=nest, random_state=seed)
    learner_prop.fit(np.vstack((X_prop_train, X_cal)),
                     np.hstack((y_prop_train, y_cal)))

    y_pred = learner_prop.predict(X_test)

    results["True_MIX"] = reg_metric(y_test, y_pred)
    results_mse["True_MIX"] = reg_metric_mse(y_test, y_pred)

    if extra_eval == True:
        results_mse_above["True_MIX"] = reg_metric_mse(
            y_test[above_ids], y_pred[above_ids]
        )
        results_mse_below["True_MIX"] = reg_metric_mse(
            y_test[below_ids], y_pred[below_ids]
        )

    ####
    print("Plain...")
    learner_prop = xgb.XGBRegressor(n_estimators=nest, random_state=seed)
    learner_prop.fit(X_prop_train, y_prop_train)

    y_pred = learner_prop.predict(X_test)

    results["True_plain"] = reg_metric(y_test, y_pred)
    results_mse["True_plain"] = reg_metric_mse(y_test, y_pred)

    if extra_eval == True:
        results_mse_above["True_plain"] = reg_metric_mse(
            y_test[above_ids], y_pred[above_ids]
        )
        results_mse_below["True_plain"] = reg_metric_mse(
            y_test[below_ids], y_pred[below_ids]
        )

    ######

    print("RESIDUALS FIT CAL...")
    learner_prop = xgb.XGBRegressor(n_estimators=nest, random_state=seed)
    learner_prop.fit(X_cal, y_cal)

    y_pred = learner_prop.predict(X_prop_train)

    residuals = y_prop_train - y_pred

    myids = np.argsort(np.abs(residuals))[0:keep]


    learner_prop = xgb.XGBRegressor(n_estimators=nest, random_state=seed)
    learner_prop.fit(X_prop_train[myids, :], y_prop_train[myids])

    y_pred = learner_prop.predict(X_test)

    results["Residuals_fit_cal"] = reg_metric(y_test, y_pred)
    results_mse["Residuals_fit_cal"] = reg_metric_mse(y_test, y_pred)

    if extra_eval == True:
        results_mse_above["Residuals_fit_cal"] = reg_metric_mse(
            y_test[above_ids], y_pred[above_ids]
        )
        results_mse_below["Residuals_fit_cal"] = reg_metric_mse(
            y_test[below_ids], y_pred[below_ids]
        )

    ######
    print("NGBOOST...")
    from ngboost import NGBRegressor
    from ngboost.distns import Normal

    base_learner = LinearRegression()

    learner_prop = NGBRegressor(Dist=Normal, Base=base_learner)
    learner_prop.fit(X_cal, y_cal)

    y_dists = learner_prop.pred_dist(X_prop_train)
    uncerts = y_dists[0:].params["scale"]

    myids = np.argsort(uncerts)[0:keep]


    learner_prop = xgb.XGBRegressor(n_estimators=nest, random_state=seed)
    learner_prop.fit(X_prop_train[myids, :], y_prop_train[myids])

    y_pred = learner_prop.predict(X_test)

    results["NGBOOST"] = reg_metric(y_test, y_pred)
    results_mse["NGBOOST"] = reg_metric_mse(y_test, y_pred)

    if extra_eval == True:
        results_mse_above["NGBOOST"] = reg_metric_mse(
            y_test[above_ids], y_pred[above_ids]
        )
        results_mse_below["NGBOOST"] = reg_metric_mse(
            y_test[below_ids], y_pred[below_ids]
        )

    print("Bayes ridge alea...")
    from sklearn.linear_model import BayesianRidge

    class ModifiedBayesianRidge(BayesianRidge):
        def predict(self, X):
            y_mean = self._decision_function(X)
            aleatoric = 1.0 / self.alpha_
            epistemic = (np.dot(X, self.sigma_) * X).sum(axis=1)
            return y_mean, aleatoric, epistemic

    learner_prop = ModifiedBayesianRidge()

    learner_prop.fit(X_cal, y_cal)

    y_dists, aleatoric, epistemic = learner_prop.predict(X_prop_train)

    myids = np.argsort(aleatoric)[0:keep]

    learner_prop = xgb.XGBRegressor(n_estimators=nest, random_state=seed)
    learner_prop.fit(X_prop_train[myids, :], y_prop_train[myids])

    y_pred = learner_prop.predict(X_test)

    results["Bayesian_ridge_aleatoric"] = reg_metric(y_test, y_pred)
    results_mse["Bayesian_ridge_aleatoric"] = reg_metric_mse(y_test, y_pred)

    if extra_eval == True:
        results_mse_above["Bayesian_ridge_aleatoric"] = reg_metric_mse(
            y_test[above_ids], y_pred[above_ids]
        )
        results_mse_below["Bayesian_ridge_aleatoric"] = reg_metric_mse(
            y_test[below_ids], y_pred[below_ids]
        )

    ########
    print("Bayes ridge...")
    from sklearn.linear_model import BayesianRidge

    learner_prop = BayesianRidge()

    learner_prop.fit(X_cal, y_cal)

    y_dists, uncerts = learner_prop.predict(X_prop_train, return_std=True)

    myids = np.argsort(uncerts)[0:keep]

    learner_prop = xgb.XGBRegressor(n_estimators=nest, random_state=seed)
    learner_prop.fit(X_prop_train[myids, :], y_prop_train[myids])

    y_pred = learner_prop.predict(X_test)

    results["Bayesian_ridge"] = reg_metric(y_test, y_pred)
    results_mse["Bayesian_ridge"] = reg_metric_mse(y_test, y_pred)

    if extra_eval == True:
        results_mse_above["Bayesian_ridge"] = reg_metric_mse(
            y_test[above_ids], y_pred[above_ids]
        )
        results_mse_below["Bayesian_ridge"] = reg_metric_mse(
            y_test[below_ids], y_pred[below_ids]
        )

    ########

    # import torch

    # from uq360.algorithms.variational_bayesian_neural_networks.bnn import BnnRegression

    # config = {
    #               "ip_dim":X_prop_train.shape[1],
    #               "op_dim":1,
    #               "num_nodes":8,
    #               "num_layers":5,
    #               "step_size":3e-2,
    #               "num_epochs":20,
    #               "random_state":seed,
    #           }

    # learner_prop = BnnRegression(config = config, prior = 'Gaussian') #, Hshoe, RegHshoe

    # learner_prop.fit(torch.Tensor(X_cal), torch.Tensor(y_cal))

    # y_dists, lower, upper, _, _ = learner_prop.predict(torch.Tensor(X_prop_train))

    # uncerts = upper-lower

    # myids = np.argsort(uncerts)[0:keep]

    # print(f'Fitting on cal size = {X_cal.shape}')
    # print(f'BNN refit = {X_prop_train[myids,:].shape}, {len(uncerts)}')

    # learner_prop = xgb.XGBRegressor(n_estimators=nest, random_state=seed)
    # learner_prop.fit(X_prop_train[myids,:], y_prop_train[myids])

    # y_pred = learner_prop.predict(X_test)

    # results['BNN'] = reg_metric(y_test, y_pred)
    # results_mse['BNN'] = reg_metric_mse(y_test, y_pred)

    # if extra_eval==True:
    #     results_mse_above['BNN'] = reg_metric_mse(y_test[above_ids], y_pred[above_ids])
    #     results_mse_below['BNN'] = reg_metric_mse(y_test[below_ids], y_pred[below_ids])

    ########

    from sklearn.gaussian_process import GaussianProcessRegressor

    gpr = GaussianProcessRegressor(random_state=seed)
    gpr.fit(X_cal, y_cal)

    y_dists, uncerts = gpr.predict(X_prop_train, return_std=True)

    myids = np.argsort(uncerts)[0:keep]


    learner_prop = xgb.XGBRegressor(n_estimators=nest, random_state=seed)
    learner_prop.fit(X_prop_train[myids, :], y_prop_train[myids])

    y_pred = learner_prop.predict(X_test)

    results["GP"] = reg_metric(y_test, y_pred)
    results_mse["GP"] = reg_metric_mse(y_test, y_pred)

    if extra_eval == True:
        results_mse_above["GP"] = reg_metric_mse(
            y_test[above_ids], y_pred[above_ids])
        results_mse_below["GP"] = reg_metric_mse(
            y_test[below_ids], y_pred[below_ids])

    if extra_eval == True:
        return results, results_mse, results_mse_above, results_mse_below

    else:
        return results, results_mse

In [4]:


full_results = {}
full_results_mse = {}
full_results_mse_above = {}
full_results_mse_below = {}
full_pvals = {}
full_errors = {}
full_crps = {}
full_cpds = {}
full_intervals = {}
full_preds = {}
full_upper = {}
full_lower = {}


cal_list = [0.01, 0.02, 0.03, 0.04, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]


n_runs = 5

for cal_size in tqdm(cal_list):
    print("--------------------------------")
    print(f"Running cal size = {cal_size}")
    print("--------------------------------")
    final_results = []
    final_results_mse = []
    final_results_mse_above = []
    final_results_mse_below = []
    for i in range(n_runs):

        seed = i * 10
        random.seed(seed)
        seed_everything(seed)

        X_prop_train, y_prop_train, _ = load_seer_cutract_dataset(
            name="seer", seed=seed
        )

        min_y = y_prop_train.min()
        max_y = y_prop_train.max()

        y_prop_train = np.array(
            [
                (y_prop_train[i] - min_y) / (max_y - min_y)
                for i in range(len(y_prop_train))
            ]
        )
        y_prop_train = pd.Series(y_prop_train)

        X_test_all, y_test_all, _ = load_seer_cutract_dataset(
            name="cutract", seed=seed)

        y_test_all = np.array(
            [(y_test_all[i] - min_y) / (max_y - min_y)
             for i in range(len(y_test_all))]
        )
        y_test_all = pd.Series(y_test_all)

        test_ids = random.sample(
            list(range(len(y_test_all))), int(0.5 * len(y_test_all))
        )

        X_test, y_test = X_test_all.iloc[test_ids,
                                         :], y_test_all.iloc[test_ids]

        remaining_eval_ids = np.setdiff1d(range(len(y_test_all)), test_ids)

        _, X_cal, _, y_cal = train_test_split(
            X_test_all.iloc[remaining_eval_ids, :],
            y_test_all.iloc[remaining_eval_ids],
            test_size=cal_size,
            random_state=seed,
        )

        X_prop_train, X_cal, X_test = (
            np.array(X_prop_train),
            np.array(X_cal),
            np.array(X_test),
        )
        y_prop_train, y_cal, y_test = (
            np.array(y_prop_train),
            np.array(y_cal),
            np.array(y_test),
        )

        prop = 0.1
        num_ids = int(prop * len(y_prop_train))
        last_ids = range(len(y_prop_train))
        y_prop_train_corrupt = deepcopy(y_prop_train)
        corruptids = random.sample(list(last_ids), num_ids)
        for myid in corruptids:
            corruption = (np.random.choice(5, 1)[0] / 2) + 1
            y_prop_train_corrupt[myid] = y_prop_train_corrupt[myid] * corruption

        nest = 10
        learner = xgb.XGBRegressor(n_estimators=nest, random_state=seed)
        learner.fit(X_prop_train, y_prop_train)

        y_eval = y_prop_train
        X_eval = X_prop_train

        triage = Triage(
            X_eval=X_eval,
            y_eval=y_eval,
            X_cal=X_cal,
            y_cal=y_cal,
            nest=nest,
            learner=learner,
        )
        groups_ids, raw_metrics = triage.run(
            compute_cpd=True, compute_crps=True)

        triage_array = raw_metrics["score_metric"]
        errors_array = raw_metrics["errors_array"]
        interval_array = raw_metrics["interval_array"]
        crps_array = raw_metrics["crps_array"]
        cpds_array = raw_metrics["cpds_array"]
        preds_array = raw_metrics["preds_array"]
        upper_array = raw_metrics["upper_array"]
        lower_array = raw_metrics["lower_array"]

        metric = triage_array

        percentile_thresh = 75
        thresh = 0.33
        conf_thresh_low = thresh
        conf_thresh_high = 1 - thresh
        conf_thresh = 0.5

        uncert = np.mean(metric * (1 - metric), axis=-1)
        confidence = np.mean(metric, axis=-1)

        # Get groups and mainly well-estimated groups
        oe_group = np.where(
            (confidence <= conf_thresh_low)
            & (uncert <= np.percentile(uncert, percentile_thresh))
        )[0]
        ue_group = np.where(
            (confidence >= conf_thresh_high)
            & (uncert <= np.percentile(uncert, percentile_thresh))
        )[0]

        combined_group = np.concatenate((oe_group, ue_group))

        we_group = []
        for id in range(len(confidence)):
            if id not in combined_group:
                we_group.append(id)

        we_group = np.array(we_group)

        groups_ids = {
            "ue_group": ue_group,
            "oe_group": oe_group,
            "we_group": we_group,
            "confidence": confidence,
            "uncert": uncert,
            "errors_array": errors_array,
            "interval_array": interval_array,
        }

        # groups = []
        # for i in range(len(triage_array)):
        #     if i in easy_train:
        #         groups.append(0)
        #     if i in ambig_train:
        #         groups.append(1)
        #     if i in hard_train:
        #         groups.append(2)

        if long_tail == True or imbalance == True:
            (
                results,
                results_mse,
                results_mse_above,
                results_mse_below,
            ) = evaluate_methods(
                X_prop_train,
                y_prop_train,
                X_cal,
                y_cal,
                X_test,
                y_test,
                seed,
                nest,
                groups_ids,
                long_tail=long_tail,
                imbalance=imbalance,
            )
            final_results_mse_above.append(results_mse_above)
            final_results_mse_below.append(results_mse_below)
        else:
            results, results_mse = evaluate_methods(
                X_prop_train,
                y_prop_train,
                X_cal,
                y_cal,
                X_test,
                y_test,
                seed,
                nest,
                groups_ids,
                long_tail=long_tail,
                imbalance=imbalance,
            )
        print(results)
        final_results.append(results)
        final_results_mse.append(results_mse)

    full_results[cal_size] = final_results
    full_results_mse[cal_size] = final_results_mse
    full_results_mse_above[cal_size] = final_results_mse_above
    full_results_mse_below[cal_size] = final_results_mse_below
    full_pvals[cal_size] = triage_array
    full_errors[cal_size] = errors_array
    full_crps[cal_size] = crps_array
    full_cpds[cal_size] = cpds_array
    full_intervals[cal_size] = interval_array
    full_preds[cal_size] = preds_array
    full_upper[cal_size] = upper_array
    full_lower[cal_size] = lower_array

  0%|          | 0/10 [00:00<?, ?it/s]

--------------------------------
Running cal size = 0.01
--------------------------------
TRIAGE WE...
0.01 - keep is 9653
TRIAGE OE ...
TRIAGE UE...
TRIAGE Combined...
CPS CONFIDENCE...
RESIDUALS DIRECT...
INTERVALS DIRECT...
Actual eval fit...
MIX fit...
Plain...
RESIDUALS FIT CAL...
NGBOOST...
[iter 0] loss=-1.2885 val_loss=0.0000 scale=1.0000 norm=0.4142
[iter 100] loss=-1.7729 val_loss=0.0000 scale=1.0000 norm=0.2875
[iter 200] loss=-2.2067 val_loss=0.0000 scale=2.0000 norm=0.4843
[iter 300] loss=-2.6070 val_loss=0.0000 scale=2.0000 norm=0.4573
[iter 400] loss=-3.2655 val_loss=0.0000 scale=4.0000 norm=0.7630
Bayes ridge alea...
Bayes ridge...
{'TRIAGE_WE': 0.13980066519022472, 'TRIAGE_OE': 0.16561378462990436, 'TRIAGE_UE': 0.339142264480533, 'TRIAGE_Combined': 0.2694896341806602, 'Confidence': 0.1445883607009222, 'Residuals_direct': 0.18414321140138062, 'Intervals_direct': 0.1708181303754328, 'True_eval_fit': 0.13650791395247464, 'True_MIX': 0.16649523636363897, 'True_plain': 0.17

 10%|█         | 1/10 [02:07<19:08, 127.64s/it]

{'TRIAGE_WE': 0.12387707152624844, 'TRIAGE_OE': 0.16103343565555417, 'TRIAGE_UE': 0.4822098450369922, 'TRIAGE_Combined': 0.18821664064606797, 'Confidence': 0.14043127651942464, 'Residuals_direct': 0.2079850582486162, 'Intervals_direct': 0.19949270036570757, 'True_eval_fit': 0.20458952980566966, 'True_MIX': 0.17706678255001904, 'True_plain': 0.1842030602902898, 'Residuals_fit_cal': 0.1875156798570715, 'NGBOOST': 0.18188896001095653, 'Bayesian_ridge_aleatoric': 0.13824583053226894, 'Bayesian_ridge': 0.20206052374230463, 'GP': 0.18221128336097364}
--------------------------------
Running cal size = 0.02
--------------------------------
TRIAGE WE...
0.02 - keep is 5879
TRIAGE OE ...
TRIAGE UE...
TRIAGE Combined...
CPS CONFIDENCE...
RESIDUALS DIRECT...
INTERVALS DIRECT...
Actual eval fit...
MIX fit...
Plain...
RESIDUALS FIT CAL...
NGBOOST...
[iter 0] loss=0.1338 val_loss=0.0000 scale=1.0000 norm=0.6359
[iter 100] loss=-0.4996 val_loss=0.0000 scale=1.0000 norm=0.4042
[iter 200] loss=-0.7588 

 20%|██        | 2/10 [04:45<19:23, 145.40s/it]

...

{'TRIAGE_WE': 0.1304317368164598, 'TRIAGE_OE': 0.16414438747160648, 'TRIAGE_UE': 0.47848682928857533, 'TRIAGE_Combined': 0.18170170709345987, 'Confidence': 0.12835328870034407, 'Residuals_direct': 0.2113890593750817, 'Intervals_direct': 0.13609962338088497, 'True_eval_fit': 0.14023636406192536, 'True_MIX': 0.1491911405482092, 'True_plain': 0.1842030602902898, 'Residuals_fit_cal': 0.12837074399790782, 'NGBOOST': 0.26606102125298403, 'Bayesian_ridge_aleatoric': 0.13824583053226894, 'Bayesian_ridge': 0.24980532725533916, 'GP': 0.2024913262216457}
--------------------------------
Running cal size = 0.4
--------------------------------
TRIAGE WE...
0.4 - keep is 5840
TRIAGE OE ...
TRIAGE UE...
TRIAGE Combined...
CPS CONFIDENCE...
RESIDUALS DIRECT...
INTERVALS DIRECT...
Actual eval fit...
MIX fit...
Plain...
RESIDUALS FIT CAL...
NGBOOST...
[iter 0] loss=-0.0472 val_loss=0.0000 scale=1.0000 norm=0.5996
[iter 100] loss=-0.3045 val_loss=0.0000 scale=2.0000 norm=1.0457
[iter 200] loss=-0.3851 va

 90%|█████████ | 9/10 [49:21<07:18, 438.76s/it]

{'TRIAGE_WE': 0.12917309763807394, 'TRIAGE_OE': 0.1660657753654669, 'TRIAGE_UE': 0.4532139381020658, 'TRIAGE_Combined': 0.18456512514866796, 'Confidence': 0.13205995793488284, 'Residuals_direct': 0.20369318374771217, 'Intervals_direct': 0.1474105045317035, 'True_eval_fit': 0.14444486920336483, 'True_MIX': 0.14872238287302525, 'True_plain': 0.1842030602902898, 'Residuals_fit_cal': 0.1343342818873312, 'NGBOOST': 0.21371179245093364, 'Bayesian_ridge_aleatoric': 0.13824583053226894, 'Bayesian_ridge': 0.29513918811221596, 'GP': 0.22331425657198736}
--------------------------------
Running cal size = 0.5
--------------------------------
TRIAGE WE...
0.5 - keep is 5788
TRIAGE OE ...
TRIAGE UE...
TRIAGE Combined...
CPS CONFIDENCE...
RESIDUALS DIRECT...
INTERVALS DIRECT...
Actual eval fit...
MIX fit...
Plain...
RESIDUALS FIT CAL...
NGBOOST...
[iter 0] loss=-0.0663 val_loss=0.0000 scale=1.0000 norm=0.5972
[iter 100] loss=-0.3238 val_loss=0.0000 scale=2.0000 norm=1.0425
[iter 200] loss=-0.3999 va

100%|██████████| 10/10 [55:47<00:00, 334.72s/it]

{'TRIAGE_WE': 0.12878091310522694, 'TRIAGE_OE': 0.165251734757434, 'TRIAGE_UE': 0.432709969769364, 'TRIAGE_Combined': 0.1919095640796037, 'Confidence': 0.1410062935493042, 'Residuals_direct': 0.20387896074629144, 'Intervals_direct': 0.1351997166241214, 'True_eval_fit': 0.1396779622595139, 'True_MIX': 0.14392266368006285, 'True_plain': 0.1842030602902898, 'Residuals_fit_cal': 0.1357841446533166, 'NGBOOST': 0.23984032775630593, 'Bayesian_ridge_aleatoric': 0.13824583053226894, 'Bayesian_ridge': 0.1975838111457434, 'GP': 0.19592180841529236}


In [5]:
full_results_mse

{0.01: [{'TRIAGE_WE': 0.05236933080330731,
   'TRIAGE_OE': 0.06612806490650713,
   'TRIAGE_UE': 0.18015867105100875,
   'TRIAGE_Combined': 0.10597062368516251,
   'Confidence': 0.055599704534975515,
   'Residuals_direct': 0.07945122238065258,
   'Intervals_direct': 0.05450543046043965,
   'True_eval_fit': 0.04759946053558559,
   'True_MIX': 0.05450334317076189,
   'True_plain': 0.058218526633944806,
   'Residuals_fit_cal': 0.05131103436253092,
   'NGBOOST': 0.05461462151521877,
   'Bayesian_ridge_aleatoric': 0.04938670937663761,
   'Bayesian_ridge': 0.06350019531139893,
   'GP': 0.041935609835997666},
  {'TRIAGE_WE': 0.049614753357674686,
   'TRIAGE_OE': 0.06946975454640328,
   'TRIAGE_UE': 0.48264292313018897,
   'TRIAGE_Combined': 0.12836152809395837,
   'Confidence': 0.0895942489680956,
   'Residuals_direct': 0.046238931627128846,
   'Intervals_direct': 0.06669077284731374,
   'True_eval_fit': 0.10763977978384938,
   'True_MIX': 0.06251582811364809,
   'True_plain': 0.06658386648422

In [6]:
def compute_results(results_type):

    cal_list = [0.01, 0.02, 0.03, 0.04, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]

    mapper = {
        "TRIAGE_WE": "TRIAGE",
        "TRIAGE_UE": "CPS UNDER",
        "TRIAGE_OE": "CPS OVER",
        "TRIAGE_Combined": "Not TRIAGE",
        "CPS_Confidence": "CPS CONF",
        "Residuals_direct": "Error Sculpt",
        "Intervals_direct": "CP Intervals Sculpt",
        "True_eval_fit": "Baseline ($\Dcal$)",
        "True_MIX": "Baseline ($\Dtrain \cup \Dcal$)",
        "True_plain": "Baseline ($\Dtrain$)",
        "Residuals_fit_cal": "RES FIT CAL",
        "NGBOOST": "NGBoost",
        "Bayesian_ridge_aleatoric": "none",
        "Bayesian_ridge": "Bayesian ridge",
        "BNN": "BNN",
        "GP": "GP",
    }

    models = list(results_type[0.01][0].keys())
    props = list(results_type.keys())

    from scipy.stats import sem

    model_means = {}
    model_stds = {}

    for model in models:

        model_scores_mean = []
        model_scores_std = []

        for prop in props:

            scores = []

            for result_dict in results_type[prop]:

                scores.append(result_dict[model])

            model_scores_mean.append(np.mean(scores))
            model_scores_std.append(sem(scores))

        model_means[model] = model_scores_mean
        model_stds[model] = model_scores_std

    triage_based = ["TRIAGE_WE", "TRIAGE_UE", "TRIAGE_OE", "TRIAGE_Combined"]

    model_based = [
        "Residuals_direct",
        "Intervals_direct",
        "Residuals_fit_cal",
        "NGBOOST",
        "Bayesian_ridge",
        "BNN",
        "GP",
    ]

    data_based = ["True_eval_fit", "True_MIX", "True_plain"]

    cal_list = np.array(cal_list)

    cal_string = ""
    for idx, val in enumerate(cal_list):
        # if idx in id_eval:
        cal_string += f" & {cal_list[idx]}"

    cal_string += " \\\\"
    print(cal_string)

    for model in triage_based:
        model_string = f"{mapper[model]}"

        if model == "BNN":
            continue

        for idx, val in enumerate(cal_list):
            # if idx in id_eval:
            round = 3
            mean_score = np.round(model_means[model][idx], round)
            std_score = np.round(model_stds[model][idx], round)
            model_string += f" & {mean_score}+-{std_score}"
        model_string += " \\\\"
        print(model_string)

    for model in data_based:
        model_string = f"{mapper[model]}"

        if model == "BNN":
            continue

        for idx, val in enumerate(cal_list):
            # if idx in id_eval:
            round = 3
            mean_score = np.round(model_means[model][idx], round)
            std_score = np.round(model_stds[model][idx], round)
            model_string += f" & {mean_score}+-{std_score}"
        model_string += " \\\\"
        print(model_string)

    for model in model_based:
        model_string = f"{mapper[model]}"

        if model == "BNN":
            continue

        for idx, val in enumerate(cal_list):
            # if idx in id_eval:
            round = 3
            mean_score = np.round(model_means[model][idx], round)
            std_score = np.round(model_stds[model][idx], round)
            model_string += f" & {mean_score}+-{std_score}"

        model_string += " \\\\"
        print(model_string)

In [7]:
compute_results(results_type=full_results_mse)

 & 0.01 & 0.02 & 0.03 & 0.04 & 0.05 & 0.1 & 0.2 & 0.3 & 0.4 & 0.5 \\
TRIAGE & 0.051+-0.003 & 0.05+-0.003 & 0.046+-0.002 & 0.046+-0.002 & 0.046+-0.001 & 0.046+-0.002 & 0.045+-0.001 & 0.045+-0.001 & 0.046+-0.002 & 0.046+-0.002 \\
CPS UNDER & 0.278+-0.054 & 0.268+-0.048 & 0.284+-0.039 & 0.28+-0.034 & 0.252+-0.02 & 0.253+-0.005 & 0.265+-0.005 & 0.282+-0.006 & 0.261+-0.01 & 0.259+-0.004 \\
CPS OVER & 0.069+-0.002 & 0.067+-0.001 & 0.067+-0.001 & 0.066+-0.001 & 0.066+-0.001 & 0.066+-0.002 & 0.065+-0.001 & 0.065+-0.002 & 0.064+-0.002 & 0.065+-0.001 \\
Not TRIAGE & 0.086+-0.013 & 0.071+-0.007 & 0.069+-0.005 & 0.064+-0.004 & 0.067+-0.005 & 0.081+-0.011 & 0.087+-0.015 & 0.094+-0.015 & 0.081+-0.012 & 0.08+-0.008 \\
Baseline ($\Dcal$) & 0.092+-0.023 & 0.07+-0.012 & 0.07+-0.01 & 0.064+-0.003 & 0.066+-0.009 & 0.052+-0.004 & 0.047+-0.001 & 0.043+-0.001 & 0.043+-0.001 & 0.041+-0.001 \\
Baseline ($\Dtrain \cup \Dcal$) & 0.06+-0.002 & 0.059+-0.002 & 0.058+-0.002 & 0.056+-0.002 & 0.055+-0.001 & 0.049+-0.0